In [4]:
!ls -la /home/azeghost/datasets/MiniImageNet

total 1240512
drwxrwxr-x 2 azeghost azeghost      4096 Oct 14 20:07 .
drwxr-xr-x 4 azeghost azeghost      4096 Oct 14 20:07 ..
-rw-rw-r-- 1 azeghost azeghost 254052267 Nov 15  2019 mini-imagenet-cache-test.pkl
-rw-rw-r-- 1 azeghost azeghost 812967380 Nov 15  2019 mini-imagenet-cache-train.pkl
-rw-rw-r-- 1 azeghost azeghost 203241803 Nov 15  2019 mini-imagenet-cache-val.pkl


In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import os
import sys
import lmdb
import pickle
import numpy as np

In [3]:
os.chdir('../') # adress to git dir
print(os.getcwd())

/home/azeghost/git/lmdb_new/LMDB_Datasets


In [36]:
!mkdir .data/imagenet_LMDB

In [ ]:
# !rm -rf .data/imagenet_LMDB

In [37]:
!ls .data/imagenet_LMDB

# Download dataset

In [7]:
train.keys()

dict_keys(['image_data', 'class_dict'])

In [35]:
# mini-imagenet-cache-test.pkl

train_in = open("/home/azeghost/datasets/MiniImageNet/mini-imagenet-cache-train.pkl", "rb")

train = pickle.load(train_in)

Xtrain = train["image_data"]

Xtrain = Xtrain.reshape([-1, 84, 84, 3])
# 64 * 600


In [46]:
import numpy as np

#Wrapper class for dataset
class DatasetWrapper:
    def __init__(self, image):
        try:
            self.channels = image.shape[2]
        except:
            self.channels = 1
        self.size = image.shape[:2]
        self.image = image.tobytes()

    def get_image(self):
        """ Returns the image as a numpy array. """
        images = np.frombuffer(self.image, dtype=np.float32) #pay attention if you  don't use create_image_lists
        return images.reshape(*self.size, self.channels) 

In [47]:
import lmdb 
def store_single_lmdb( filename, img, index,  num_images):
    """ Stores a wrapper to LMDB.
    """
    map_size = num_images * img.nbytes * 10
    env = lmdb.open(filename, map_size=map_size)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
        # All key-value pairs need to be Strings
        value = DatasetWrapper(img)
        key = f"{index:08}"
        txn.put(key.encode("ascii"), pickle.dumps(value))

    env.close()

In [63]:
category = 'train'
scaler = 255.0
images = Xtrain
lmdb_dir = ".data/imagenet_LMDB"

In [65]:
from tqdm import tqdm

# create_if_not_exist(lmdb_dir)
num_images = images.shape[0]
lmdb_name = lmdb_dir + os.sep + '_{}'.format(category)
index = 0
for idx, (image) in tqdm(enumerate(images),total=num_images):
                img = np.float32(image) / scaler

                store_single_lmdb(index=index, filename=lmdb_name, img=img,
                                       num_images=num_images)
                index = index + 1


100%|██████████| 38400/38400 [01:59<00:00, 320.91it/s]


# Read the lmdb 

#Todo
 - Env put outside of loops for write and read
 - Try the Dynamic code again with clean kernel 
 - Create Standartize as an option

In [ ]:
def read_lmdb(lmdb_dir, num_images):
    images = []
    env = lmdb.open(lmdb_dir, readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))

            dataset = pickle.loads(data)
            images.append(dataset.get_image())
            labels_list = [attr for attr in dir(dataset) if
                           not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and
                           (not attr in ['image', 'channels', 'size'])]

            for label in labels_list:
                # _lab = {label: eval(f'dataset.{label}')}
                # labels = {**labels, **_lab}
                if label in labels:
                    labels[label].append(eval(f'dataset.{label}'))
                else:
                    labels = {label: [eval(f'dataset.{label}')] }


    env.close()
    return {'images': images, **labels}

# Tar and unzip LMDB

In [ ]:
import os
os.chdir('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data')
# !ls -la

In [ ]:
!tar -cvjf mnist.tar.bz MNIST_LMDB/* 

In [ ]:
!mv mnist.tar.bz /home/azeghost/git/lmdb_new/LMDB_Datasets/.data/MNIST_TAR

In [ ]:
!ls /home/azeghost/git/lmdb_new/LMDB_Datasets/.data/MNIST_TAR

In [ ]:
# !split -b 50M mnist.tar.bz "mnist.tar.part"
#split -b <max size> <name of zip or dir to zip/name> <split file name beginning>

#Check if they are created
# !ls -lh mnist.tar.part*

In [ ]:
# Combine Them

#pokemon_training
# !cat mnist.tar.part* > mnist_combined.tar.bz
#!cat <split files put * at the end> > <final zip name>
# !ls -la 

!mkdir ./.test_Mnist

!tar -xf mnist.tar.bz --directory ./.test_Mnist

# !rm -rf ./.test

In [ ]:
!ls -la ./.test_Mnist/MNIST_LMDB

## LMDBImageIterator and LMDBImageGenerator

In [ ]:
os.chdir('/home/azeghost/git/Generative_Models/')

In [ ]:
os.getcwd()

In [ ]:
import os
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img
from keras import backend as K
import logging
from utils.reporting.logging import log_message
from utils.data_and_files.file_utils import get_file_path

class LMDBImageIterator(Iterator):


    def __init__(self,
                 num_images,
                 category,
                 lmdb_dir,
                 batch_size,
                 episode_len=20,
                 episode_shift=10,
                 shuffle=True,
                 seed=None,
                 save_to_dir=None,
                 save_prefix='',
                 save_format='jpeg',
                 dtype=K.floatx(),
                 ):
        
        self.category = category
        self.batch_size = batch_size
        
        
        self.lmdb_dir = lmdb_dir
        self.episode_len = episode_len
        self.episode_shift = episode_shift

        
        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format
        print("Initializing Iterator " + category +" Number of images " +str(num_images))
        print(category,lmdb_dir, batch_size,shuffle,seed)
        self.env = lmdb.open(lmdb_dir, readonly=True)
        
        Iterator.__init__(self, num_images, batch_size, shuffle, seed)
        
        
    def __del__(self):
        self.env.close()
        
        
    def _get_batches_of_transformed_samples(self, index_array):
        print(index_array)
        images, labels = [], []
        
        if len(index_array) < self.batch_size:
            diff = self.batch_size//len(index_array) + 1
            index_array = np.repeat(index_array, diff, axis=0)[:self.batch_size]

        else:
                with self.env.begin() as txn:
                    for image_id in index_array:
                        data = txn.get(f"{image_id:08}".encode("ascii"))
                        dataset = pickle.loads(data)
                        images.append(dataset.get_image())
                        labels_list = [attr for attr in dir(dataset) if not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and 
                           (not attr in ['image','channels',  'size'] )]

                        for label in labels_list:
                            _lab = {label: eval(f'dataset.{label}')}
                            labels = {**labels, **_lab}
        return {'images': images, **labels}


In [ ]:
import glob
import hashlib
import logging
import os
import re
import warnings

from keras.preprocessing.image import ImageDataGenerator

from utils.data_and_files.data_utils import as_bytes
from utils.reporting.logging import log_message


class LMDBImageGenerator(ImageDataGenerator):
    def flow_from_lmdb_lists(self, 
                              num_images,
                              category,
                              lmdb_dir,
                              batch_size,
                              episode_len=None,
                              episode_shift=None,
                              color_mode='rgb',
                              shuffle =True,
                              seed=None
                              ):

        
          

        return LMDBImageIterator(
                             num_images = num_images,
                             category = category,
                             lmdb_dir = lmdb_dir,
                             batch_size  = batch_size,
                             episode_len = episode_len,
                             episode_shift =episode_shift,
                             shuffle = shuffle,
                             seed = seed)


def get_generators( val_lmdb_dir, val_num_images, tra_lmdb_dir, tra_num_images, 
                   batch_size, episode_len=None, episode_shift=None):

    train_datagen = LMDBImageGenerator()

    valid_datagen = LMDBImageGenerator()

    train_generator = train_datagen.flow_from_lmdb_lists(
        num_images = tra_num_images,
        category='training',
        lmdb_dir=tra_lmdb_dir,
        batch_size=batch_size,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    validation_generator = valid_datagen.flow_from_lmdb_lists(
        num_images = val_num_images,
        category='validation',
        lmdb_dir=val_lmdb_dir,
        batch_size=batch_size,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    return train_generator, validation_generator



In [ ]:
training_generator, testing_generator = get_generators(
    val_lmdb_dir = '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_validation',
    val_num_images = 218,
    tra_lmdb_dir = '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_training', 
    tra_num_images = 591,
    batch_size=100
)

In [ ]:
x, y = training_generator.next()

In [ ]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

In [ ]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

In [ ]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator,     
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break